# task 3 : Generate the time series dataset (this step can be skipped; you can directly download the converted dataset).

1. Convert pcap to JSON format (to prevent the pcap file from being too large, use editcap to split the pcap into multiple smaller files, and use tshark to convert the pcap to JSON format).
Download link: https://huggingface.co/datasets/JimXie/IIoTset/blob/main/pcap-json.tar.gz

2. Add labels to the JSON files to generate the time series dataset.
Download link: https://huggingface.co/datasets/JimXie/IIoTset/blob/main/dataset.tar.gz

In [1]:
import os,json,sys,logging
sys.path.append("./share")
sys.path.append("./common")
import pandas as pd
import numpy as np
from IoTCommon import CIoTCommon
from Config import g_data_root
from tqdm.notebook import tqdm
import warnings,os,json,csv
import os
from Config import g_data_root
warnings.simplefilter("ignore")
tqdm.pandas()

# Pcap format to json format

In [3]:
g_attack = g_data_root+"/raw/Attack traffic"
g_normal = g_data_root+"/raw/Normal traffic"
g_temp = g_data_root+"/raw/temp"
g_output = g_data_root+"/pcap-json"

def get_pcap_files(directory):
    ret = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.pcap'):
                file_name = "%s/%s" % (root, file)
                ret.append(file_name)
    return ret
    
def get_temp_files():
    directory = g_temp
    ret = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.startswith('test_'):
                file_name = "%s/%s" % (root, file)
                ret.append(file_name)
    return ret
def split_pcap(file_name):
    cmd = 'editcap -c 1024 "%s" "%s"/test'%(file_name,g_temp)
    os.system(cmd)
    print(cmd)
    
def convert_pcap(temp_file,json_file):
    cmd = 'tshark -T json -r "%s" > "%s"'%(temp_file, json_file)
    os.system(cmd)
    print(cmd)

def pcap2json(file_name):
    for temp_file in get_temp_files():
        if temp_file.find('test_') > 0 :
            os.remove(temp_file)
    if len(get_temp_files() ) > 0:
        print("can't clear temp files")
        return
        
    split_pcap(file_name)
    t_name = file_name.split("/")[-1].split(".pcap")[0]
    temp_files = get_temp_files()
    os.system('mkdir -p "%s/%s"'%(g_output,t_name))
              
    for i in range(len(temp_files)):
        temp_file = temp_files[i]
        json_file = "%s/%s/%d.json"%(g_output,t_name,i)
        convert_pcap(temp_file, json_file)

In [ ]:
#covert attack traffic
for file_name in get_pcap_files(g_attack):
    pcap2json(file_name)
#covert IoT sensor traffic
for file_name in get_pcap_files(g_normal):
    pcap2json(file_name)

# Append labels to json format data and create time series dataset.

In [4]:
g_attack_ip = {
    "Backdoor_attack":["192.168.0.170"],
    "Password attacks":["192.168.0.170"],
    "Ransomware attack":["192.168.0.170"],
    "Uploading attack":["192.168.0.170"],
    "SQL injection attack":["192.168.0.170"],
    "XSS attacks":["192.168.0.170","172.217.19.42","104.16.87.20"],
    "MITM (ARP spoofing + DNS) Attack":["192.168.0.101","192.168.0.152","172.217.19.35","192.168.0.170"],
    "Vulnerability scanner attack":["192.168.0.170","142.250.200.205","172.217.19.35","142.250.201.10"],
    "OS Fingerprinting attack":["192.168.0.170"],
    "Port Scanning attack":["192.168.0.170"],
    "DDoS ICMP Flood Attacks":["213.117.18.213","183.223.100.122","166.153.227.121","49.81.59.152","227.117.33.125"],
    "DDoS HTTP Flood Attacks":["192.168.0.170","216.58.198.74"],
    "DDoS UDP Flood Attacks":["190.123.219.128","16.226.184.201","153.125.214.15","91.184.12.91"],
    "DDoS TCP SYN Flood Attacks":["207.192.25.133","94.196.109.185","133.149.252.77","220.146.94.148"]
}
g_ip_mac_map = {
    "DDoS TCP SYN Flood Attacks":{
        "207.192.25.133":"d8:f2:ca:8e:17:69",
        "94.196.109.185":"d8:f2:ca:8e:17:69",
        "133.149.252.77":"d8:f2:ca:8e:17:69",
        "220.146.94.148":"d8:f2:ca:8e:17:69"
    },
    'DDoS UDP Flood Attacks':{
        "190.123.219.128":"d8:f2:ca:8e:17:69",
        "16.226.184.201":"d8:f2:ca:8e:17:69",
        "153.125.214.15":"d8:f2:ca:8e:17:69",
        "91.184.12.91":"d8:f2:ca:8e:17:69"
    },
    "DDoS HTTP Flood Attacks":{
        "192.168.0.170":"d8:f2:ca:8e:17:69",
        "216.58.198.74":"48:f9:7c:f6:cb:a1"
    },
    "DDoS ICMP Flood Attacks":{
        "213.117.18.213":"d8:f2:ca:8e:17:69",
        "183.223.100.122":"d8:f2:ca:8e:17:69",
        "166.153.227.121":"d8:f2:ca:8e:17:69",
        "49.81.59.152":"d8:f2:ca:8e:17:69",
        "227.117.33.125":"d8:f2:ca:8e:17:69"
    },
    "Port Scanning attack":{
        "192.168.0.170":"d8:f2:ca:8e:17:69"
    },
    "OS Fingerprinting attack":{
        "192.168.0.170":"d8:f2:ca:8e:17:69"
    },
    "Vulnerability scanner attack":{
        "192.168.0.170":"d8:f2:ca:8e:17:69",
        "142.250.200.205":"48:f9:7c:f6:cb:a1",
        "172.217.19.35":"48:f9:7c:f6:cb:a1",
        "142.250.201.10":"48:f9:7c:f6:cb:a1"
    },
    "MITM (ARP spoofing + DNS) Attack":{
        "192.168.0.101":"d8:f2:ca:8e:17:69",
        "192.168.0.152":"xx:xx:xx:xx:xx:xx",
        "172.217.19.35":"xx:xx:xx:xx:xx:xx",
        "192.168.0.170":"xx:xx:xx:xx:xx:xx"
    },
    "XSS attacks":{
        "192.168.0.170":"d8:f2:ca:8e:17:69",
        "172.217.19.42":"48:f9:7c:f6:cb:a1",
        "104.16.87.20":"48:f9:7c:f6:cb:a1"
    },
    "SQL injection attack":{
        "192.168.0.170":"d8:f2:ca:8e:17:69"
    },
    "Uploading attack":{
        "192.168.0.170":"d8:f2:ca:8e:17:69"
    }, 
    "Backdoor_attack":{
        "192.168.0.170":"d8:f2:ca:8e:17:69"
    },
    "Password attacks":{
        "192.168.0.170":"d8:f2:ca:8e:17:69"
    },
    "Ransomware attack":{
        "192.168.0.170":"d8:f2:ca:8e:17:69"
    }
}
g_data_set = g_data_root + "dataset/"

In [ ]:
for fi in tqdm(CIoTCommon.get_json_files("%spcap-json/"%g_data_root)):
    fi = fi.replace("\\","/")
    Category,AttackType,file_name = fi.split("/")[-3],fi.split("/")[-2],fi.split("/")[-1]
    directory = os.path.dirname(fi)
    directory = directory.replace("%s"%g_data_root,g_data_set).replace("/pcap-json","")
    new_file_name = "%s/%s"%(directory, file_name)
    #new_file_name = new_file_name.replace(".json",".csv")
    
    if Category == "Attack":
        ip_list = g_attack_ip[AttackType]
    else:
        ip_list = []        
    data = CIoTCommon.get_feature_data(fi)
    if data:
        df_tmp = pd.DataFrame(data)
        if Category == "Attack":
            withIp = False
            withArp = False
            mac_list = list(g_ip_mac_map[AttackType].values())
            #IP协议
            if 'ip.src' in df_tmp.keys().tolist():
                withIp = True
                if AttackType.find("Flood Attacks") > 0:
                    mask_ip_send = df_tmp['eth.src'].isin(mac_list)
                    mask_ip_recv = df_tmp['eth.dst'].isin(mac_list)
                else:
                    mask_ip_send = df_tmp['ip.src'].isin(ip_list) 
                    mask_ip_recv = df_tmp['ip.dst'].isin(ip_list)
            #ARP协议
            if 'arp.src.proto_ipv4' in df_tmp.keys().tolist(): 
                withArp = True
                if AttackType.find("Flood Attacks") > 0:
                    mask_arp_send = df_tmp['eth.src'].isin(mac_list)
                    mask_arp_recv = df_tmp['eth.dst'].isin(mac_list)
                else:
                    mask_arp_send = df_tmp['arp.src.proto_ipv4'].isin(ip_list) 
                    mask_arp_recv = df_tmp['arp.dst.proto_ipv4'].isin(ip_list)
                
            if withIp:
                if withArp:
                    mask_send = (mask_ip_send | mask_arp_send)
                    mask_recv = (mask_ip_recv | mask_arp_recv)
                else:
                    mask_send = mask_ip_send
                    mask_recv = mask_ip_recv
                    
                df_tmp.loc[mask_send,"Label"] = '1'
                df_tmp.loc[mask_recv,"Label"] = '-1'
                df_tmp.loc[~(mask_send|mask_recv),"Label"] = '0'
                df_tmp["Label"].astype(int)
            else:
                if withArp:
                    mask_send = mask_arp_send
                    mask_recv = mask_arp_recv
                    
                    df_tmp.loc[mask_send,"Label"] = '1'
                    df_tmp.loc[mask_recv,"Label"] = '-1'
                    df_tmp.loc[~(mask_send|mask_recv),"Label"] = '0'
                    df_tmp["Label"].astype(int)
                else:
                    df_tmp["Label"] = "0" 
                    df_tmp["Label"].astype(int)
        else:
            df_tmp["Label"] = "0"
            df_tmp["Label"].astype(int)
            
        os.makedirs(directory, exist_ok=True)
        df_tmp.to_json(new_file_name,orient='records')
        #df_tmp.to_csv(new_file_name,index=False,sep=',', quotechar='"', quoting=csv.QUOTE_MINIMAL, doublequote=True, encoding='utf-8')